In [50]:
import requests 
import pandas as pd 
from datetime import date
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
import time 
import json
import numpy as np
import plotly as pt 

In [59]:
todays_date=date.today()
current_year=todays_date.year
years=list(range(1990,current_year))


In [6]:
init_url="https://www.basketball-reference.com/awards/awards_{}.html"

In [14]:
# Web scrapping: MPV awards voting stats from 1990 to 2021
for year in years:
    url=init_url.format(year)
    data_yearly_awards=requests.get(url)

    with open("yearly_awards/awards_{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(data_yearly_awards.text)


In [60]:
# HTML parsing from downloaded HTML files 
list_dfs=[]
for year in years:
    with open("yearly_awards/awards_{}.html".format(year),encoding="utf-8") as f:
        page=f.read()
    soup=bs(page,"html.parser")
    soup.find("tr", class_="over_header").decompose()
    mvp_table=soup.find(id="mvp")
    mvp_df=pd.read_html(str(mvp_table))[0]
    mvp_df['Year']=year
    list_dfs.append(mvp_df)



In [61]:
index=list(range(0,len(years)))
zipped=list(zip(years,index))
index_dict={year:index for (year,index) in zipped}


In [62]:
mvp_voting_table=pd.concat(list_dfs)
mvp_voting_table[mvp_voting_table["Year"]==2021].iloc[0]

Rank                  1
Player     Nikola Jokić
Age                  25
Tm                  DEN
First              91.0
Pts Won           971.0
Pts Max            1010
Share             0.961
G                    72
MP                 34.6
PTS                26.4
TRB                10.8
AST                 8.3
STL                 1.3
BLK                 0.7
FG%               0.566
3P%               0.388
FT%               0.868
WS                 15.6
WS/48             0.301
Year               2021
Name: 0, dtype: object

In [63]:
mvp_voting_table.to_csv("mvp_votings.csv")

In [64]:
mvp_votings=pd.read_csv("mvp_votings.csv")
mvp_votings=mvp_votings.drop("Unnamed: 0",axis=1)

In [65]:
mvp_votings

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Magic Johnson,30,LAL,27.0,636.0,920,0.691,79,37.2,...,6.6,11.5,1.7,0.4,0.480,0.384,0.890,16.5,0.270,1990
1,2,Charles Barkley,26,PHI,38.0,614.0,920,0.667,79,39.1,...,11.5,3.9,1.9,0.6,0.600,0.217,0.749,17.3,0.269,1990
2,3,Michael Jordan,26,CHI,21.0,564.0,920,0.613,82,39.0,...,6.9,6.3,2.8,0.7,0.526,0.376,0.848,19.0,0.285,1990
3,4,Karl Malone,26,UTA,2.0,214.0,920,0.233,82,38.1,...,11.1,2.8,1.5,0.6,0.562,0.372,0.762,15.9,0.245,1990
4,5,Patrick Ewing,27,NYK,1.0,162.0,920,0.176,82,38.6,...,10.9,2.2,1.0,4.0,0.551,0.250,0.775,13.5,0.205,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,11,Russell Westbrook,32,WAS,0.0,5.0,1010,0.005,65,36.4,...,11.5,11.7,1.4,0.4,0.439,0.315,0.656,3.7,0.075,2021
484,12,Ben Simmons,24,PHI,0.0,3.0,1010,0.003,58,32.4,...,7.2,6.9,1.6,0.6,0.557,0.300,0.613,6.0,0.153,2021
485,13T,James Harden,31,TOT,0.0,1.0,1010,0.001,44,36.6,...,7.9,10.8,1.2,0.8,0.466,0.362,0.861,7.0,0.208,2021
486,13T,LeBron James,36,LAL,0.0,1.0,1010,0.001,45,33.4,...,7.7,7.8,1.1,0.6,0.513,0.365,0.698,5.6,0.179,2021


In [36]:

""" 
Rank:MVP ranking 
Player: Player name 
Age:Player's age on February 1st of the season 
Tm:Team 
First: First place votes 
pts Won :Points won 
Pts Max : Max points 


"""
mvp_votings.columns

Index(['Rank', 'Player', 'Age', 'Tm', 'First', 'Pts Won', 'Pts Max', 'Share',
       'G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS',
       'WS/48', 'Year'],
      dtype='object')

In [29]:
mvp_votings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Rank     488 non-null    object 
 1   Player   488 non-null    object 
 2   Age      488 non-null    int64  
 3   Tm       488 non-null    object 
 4   First    488 non-null    float64
 5   Pts Won  488 non-null    float64
 6   Pts Max  488 non-null    int64  
 7   Share    488 non-null    float64
 8   G        488 non-null    int64  
 9   MP       488 non-null    float64
 10  PTS      488 non-null    float64
 11  TRB      488 non-null    float64
 12  AST      488 non-null    float64
 13  STL      488 non-null    float64
 14  BLK      488 non-null    float64
 15  FG%      488 non-null    float64
 16  3P%      477 non-null    float64
 17  FT%      488 non-null    float64
 18  WS       488 non-null    float64
 19  WS/48    488 non-null    float64
 20  Year     488 non-null    int64  
dtypes: float64(14), 

In [32]:
mvp_votings[mvp_votings['3P%'].isna()==True]

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
126,13T,Dikembe Mutombo,30,ATL,0.0,4.0,1150,0.003,80,37.2,...,11.6,1.4,0.6,3.3,0.527,NaN,0.705,11.3,0.183,1997
137,4,Shaquille O'Neal,25,LAL,1.0,311.0,1160,0.268,60,36.3,...,11.4,2.4,0.7,2.4,0.584,NaN,0.527,10.2,0.224,1998
205,15T,Anthony Mason,34,MIA,0.0,1.0,1240,0.001,80,40.7,...,9.6,3.1,1.0,0.3,0.482,NaN,0.781,11.6,0.171,2001
223,16T,Elton Brand,22,LAC,0.0,1.0,1260,0.001,80,37.8,...,11.6,2.4,1.0,2.0,0.527,NaN,0.742,13.6,0.216,2002
229,5,Shaquille O'Neal,30,LAL,3.0,126.0,1190,0.106,67,37.8,...,11.1,3.1,0.6,2.4,0.574,NaN,0.622,13.2,0.250,2003
243,6,Shaquille O'Neal,31,LAL,0.0,178.0,1230,0.145,67,36.8,...,11.5,2.9,0.5,2.5,0.584,NaN,0.490,9.9,0.192,2004
255,2,Shaquille O'Neal,32,MIA,58.0,1032.0,1270,0.813,73,34.1,...,10.4,2.7,0.5,2.3,0.601,NaN,0.461,11.0,0.211,2005
267,14T,P.J. Brown,35,NOH,0.0,1.0,1270,0.001,82,34.4,...,9.0,2.2,0.9,0.6,0.446,NaN,0.864,6.9,0.118,2005
289,9T,Carlos Boozer,25,UTA,0.0,7.0,1290,0.005,74,34.6,...,11.7,3.0,0.9,0.3,0.561,NaN,0.685,9.9,0.185,2007
292,12T,Shaquille O'Neal,34,MIA,0.0,3.0,1290,0.002,40,28.4,...,7.4,2.0,0.2,1.4,0.591,NaN,0.422,2.8,0.119,2007


In [5]:
#init Selenium driver 
driver= webdriver.Chrome(executable_path="chromedriver")

In [9]:
pergame_stats_url="https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
for year in years:
    url=pergame_stats_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)

    html=driver.page_source
    with open("per_game_stats/stats_{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(html)

In [12]:
with open('../SQLcredentials.txt') as f:
    data = json.loads(f.read())

username=data['username']
password=data['password']

In [67]:
#Connect to Azure SQL database
import pyodbc

server = "tcp:firstserver.database.windows.net,1433"
database = "NBAprediction"
username = username
password = password
driver= '{ODBC Driver 18 for SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Connection Timeout=30')
cursor = cnxn.cursor()

In [68]:
#Sample select query
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

Microsoft SQL Azure (RTM) - 12.0.2000.8 
	May 12 2022 23:11:24 
	Copyright (C) 2022 Microsoft Corporation

